In [ ]:
# Lightning related imports

# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning fsspec --no-deps --target=$nb_path
# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning
!pip install torchtext==0.8.0 torch==1.7.0 torchvision==0.8 pytorch-lightning==1.2.2
!pip install neptune-client==0.5.1

# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning
# https://github.com/PyTorchLightning/pytorch-lightning/issues/6210

     |████████████████████████████████| 7.0MB 8.1MB/s 
     |████████████████████████████████| 776.8MB 15kB/s 
     |████████████████████████████████| 11.8MB 31.2MB/s 
     |████████████████████████████████| 819kB 49.1MB/s 
     |████████████████████████████████| 112kB 20.9MB/s 
     |████████████████████████████████| 276kB 61.8MB/s 
     |████████████████████████████████| 1.3MB 45.6MB/s 
     |████████████████████████████████| 143kB 49.7MB/s 
     |████████████████████████████████| 296kB 58.1MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=d372baf80f27dbdff041fac611d2cc7e6e4958796cb0c427585d59be10886899
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
ERROR: pytorch-lightning 1.2.2 has requirement future>=0.17.1, but you'll have future 0.16.0 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
 

In [ ]:
# !pip install torch==1.7.0 torchvision==0.8 torchtext==0.8
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric
!pip install ogb

     |████████████████████████████████| 2.8MB 9.3MB/s 
     |████████████████████████████████| 1.6MB 7.5MB/s 
     |████████████████████████████████| 194kB 7.8MB/s 
     |████████████████████████████████| 235kB 14.6MB/s 
     |████████████████████████████████| 2.2MB 15.0MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 61kB 4.4MB/s 
  Created wheel for outdated: filename=outdated-0.2.0-cp37-none-any.whl size=4962 sha256=42fb136f328746866be177631d052e9d0e4413e99bc36885e1caf916e63ca736
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp37-none-any.whl size=7051 sha256=f5fb68432406fa9e5ea4043e19ebc9a3264b182650bfe26f8d970fe53f8f388f
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built outdated littleutils


In [101]:
%load_ext autoreload
%autoreload 2
import pytorch_lightning
print(pytorch_lightning.__version__)
from pytorch_lightning import Trainer
from pytorch_lightning.loggers.neptune import NeptuneLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

import json
import torch
import numpy as np
import matplotlib.pyplot as plt
# from scikitplot.metrics import plot_confusion_matrix

from lightning_model import LightningPAN


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1.2.2


In [102]:
# Testing if the lightning_model script gets refreshed correctly
LightningPAN.training_epoch_end("hello",'Hle')


done?


TypeError: ignored

In [ ]:

# loading params
with open('parameters.json') as json_file:
    parameters = json.load(json_file)
print(parameters)


datasetname = parameters["dataset_name"]
phi = parameters["phi"]
runs = parameters["runs"]
batch_size = parameters["batch_size"]
filter_size = parameters["maximum_path_size"]+1
learning_rate = parameters["learning_rate"]
weight_decay = parameters["weight_decay"]
pool_ratio = parameters["pool_ratio"] # fraction of nodes to use for pooled feature array
pan_pool_weight = parameters['pan_pool_weight'] # importance weight on global matrix X
criterion_pos_weight = parameters['criterion_pos_weight'] # Over-weightage on positive labels
nhid = parameters["nhid"]
epochs = parameters['epochs']
LearningRateMonitor_Params = {'logging_interval': 'epoch'}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

model = LightningPAN(9, 1, nhid=nhid, 
                     ratio=pool_ratio, pan_pool_weight=pan_pool_weight,
                     criterion_pos_weight=criterion_pos_weight,
                     filter_size=filter_size)
lr_logger = LearningRateMonitor(**LearningRateMonitor_Params)
neptune_logger = NeptuneLogger(
                api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMDk5YjVmYzYtNTU0My00MzhkLWJiYTAtMGM4ZGVhZmEyMTZiIn0=",
                project_name='hvergnes/PAN',
                close_after_fit=False,
                params=parameters, # your hyperparameters, immutable
                tags=['PAN', 'parameterize most things', 'fix epoch end', 'change parameters'],  # tags
                upload_source_files=["parameters.json", "lightning_model.py"]
                )

trainer = Trainer(
    max_epochs=epochs,
    logger=neptune_logger,
    callbacks=[lr_logger],
    gpus=1,
    # fast_dev_run=True,
)

model.cuda()
trainer.fit(model)
trainer.test(model)

NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: None, using: 0 TPU cores


{'dataset_name': 'ogbg-molhiv', 'phi': 0.3, 'runs': 1, 'num_workers': 0, 'batch_size': 64, 'maximum_path_size': 4, 'learning_rate': 0.001, 'weight_decay': 0.001, 'pool_ratio': 0.6, 'pan_pool_weight': 0.6, 'criterion_pos_weight': 25, 'nhid': 128, 'epochs': 100}
Device: cuda
https://ui.neptune.ai/hvergnes/PAN/e/PAN-138



   | Name         | Type              | Params
----------------------------------------------------
0  | criterion    | BCEWithLogitsLoss | 0     
1  | atom_encoder | AtomEncoder       | 5.5 K 
2  | conv1        | PANConv           | 4.2 K 
3  | norm1        | Norm              | 384   
4  | pool1        | PANPooling        | 135   
5  | drop1        | PANDropout        | 0     
6  | conv2        | PANConv           | 16.5 K
7  | norm2        | Norm              | 384   
8  | pool2        | PANPooling        | 133   
9  | drop2        | PANDropout        | 0     
10 | conv3        | PANConv           | 16.5 K
11 | norm3        | Norm              | 384   
12 | pool3        | PANPooling        | 133   
13 | lin1         | Linear            | 8.3 K 
14 | bn1          | BatchNorm1d       | 128   
15 | lin2         | Linear            | 2.1 K 
16 | bn2          | BatchNorm1d       | 64    
17 | lin3         | Linear            | 33    
----------------------------------------------------


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: RuntimeWarning: You are using `LearningRateMonitor` callback with models that have no learning rate schedulers. Please see documentation for `configure_optimizers` method.
  warnings.warn(*args, **kwargs)


done?


done?


In [99]:
import os
print(os.listdir())
torch.save(model.state_dict(), 'model.pt')
print(os.listdir())

['.config', 'parameters.json', 'Norm.py', 'model.py', 'dataset', 'Untitled', '__pycache__', 'lightning_model.py', 'sample_data']
['.config', 'parameters.json', 'Norm.py', 'model.pt', 'model.py', 'dataset', 'Untitled', '__pycache__', 'lightning_model.py', 'sample_data']


In [ ]:
from google.colab import files
files.download('model.pt') 

In [ ]:
# Command to keep colab running (run on browser console (i.e. chrome, cmd+shift+c))
# function ClickConnect(){
#     console.log("Clicked on connect button"); 
#     document.querySelector("colab-connect-button").click()
# }
# setInterval(ClickConnect,60000)

# var startClickConnect = function startClickConnect(){
#     var clickConnect = function clickConnect(){
#         console.log("Connnect Clicked - Start");
#         document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
#         console.log("Connnect Clicked - End"); 
#     };

#     var intervalId = setInterval(clickConnect, 60000);

#     var stopClickConnectHandler = function stopClickConnect() {
#         console.log("Connnect Clicked Stopped - Start");
#         clearInterval(intervalId);
#         console.log("Connnect Clicked Stopped - End");
#     };

#     return stopClickConnectHandler;
# };

# var stopClickConnect = startClickConnect();

# stopClickConnect();
